#  Домашнее задание по NLP # 3 [100 баллов] 
## Классификация и генерация фамилий 

В этом домашнем задании вам предстоит классифицировать и генерировать фамилии на 19 разных языках. Обучающие данные хранятся в папке data и разбиты по языкам: один язык – одна подпапка. Ниже представлен код для считывания данных в словарь вида: 
```d{язык} : [список имен]```.

In [124]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob

def findFiles(path): return glob.glob(path)

print(findFiles('data/names/*.txt'))


def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return lines

all_categories = []
category_lines = {}
for filename in findFiles('data/names/*.txt'):
    category = filename.split('/')[-1].split('.')[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines

n_categories = len(all_categories)

print(category_lines['Russian'][100:120])

['data/names/Arabic.txt', 'data/names/Chinese.txt', 'data/names/Czech.txt', 'data/names/Dutch.txt', 'data/names/English.txt', 'data/names/French.txt', 'data/names/German.txt', 'data/names/Greek.txt', 'data/names/Irish.txt', 'data/names/Italian.txt', 'data/names/Japanese.txt', 'data/names/Korean.txt', 'data/names/Polish.txt', 'data/names/Portuguese.txt', 'data/names/Russian.txt', 'data/names/Scottish.txt', 'data/names/Spanish.txt', 'data/names/Vietnamese.txt']
['Abels', 'Abelsky', 'Abeltsev', 'Abelyan', 'Aberson', 'Abertasov', 'Abesadze', 'Abezgauz', 'Abgaryan', 'Abibulaev', 'Abidoff', 'Abidov', 'Abih', 'Abikh', 'Abisaloff', 'Abisalov', 'Abitoff', 'Abitov', 'Abjaliloff', 'Abjalilov']


## Часть 1. Предварительная обработка данных [10 баллов]

1. Удалите неоднозначные фамилии (т.е. одинаковые фамилии на разных языка), если такие есть;
2. Оцените 
* среднюю длину фамилии по всей коллекции
* по каждому языку
3. Для последующей классификации (части 2 и 3) оздайте обучающее и тестовое множество так, чтобы в обучающем множестве классы были сбалансированы: то есть, в обучающее множество должно входить примерно одинаковое количество фамилий на разных ящыка. 

In [125]:
for key in category_lines:
    print(key, len(category_lines[key]))

Arabic 2000
Chinese 268
Czech 519
Dutch 297
English 3668
French 277
German 724
Greek 203
Irish 232
Italian 709
Japanese 991
Korean 94
Polish 139
Portuguese 74
Russian 9408
Scottish 100
Spanish 298
Vietnamese 73


Для начала попробуем поработать только с русскими и английскими фамилиями

In [126]:
import numpy as np

In [127]:
russian = category_lines["Russian"]
english = category_lines["English"]

In [128]:
russian_cleared = [x for x in russian if x not in english]
english_cleared = [x for x in english if x not in russian]

In [129]:
min_len = min(len(russian_cleared), len(english_cleared))

In [130]:
min_len

3641

In [131]:
russian = russian_cleared[:min_len]
english = english_cleared[:min_len]

In [132]:
np.random.shuffle(russian)
np.random.shuffle(english)

In [133]:
print(russian[:10])
print('russian len: ', len(russian))
print(english[:10])
print('english len: ', len(english))

['Gulamov', 'Eideman', 'Filippovich', 'Baburkin', 'Bessogonov', 'Agababoff', 'Aweritchkin', 'Glasko', 'Awinowitski', 'Baikowski']
russian len:  3641
['Hunt', 'Graves', 'Donohoe', 'Conroy', 'Friel', 'Booth', 'Arnall', 'Eglan', 'Flowers', 'Allwood']
english len:  3641


In [134]:
coef = int(min_len * 0.8)
print(coef)

2912


In [135]:
zeros = np.zeros(min_len)
ones = np.ones(min_len)

Объединим английские и русские фамилии, одновременно присвоив им метку

In [137]:
russian_train = np.vstack((russian[:coef], zeros[:coef])).T
print(russian_train[:10])
print(russian_train.shape)
russian_test = np.vstack((russian[coef : ], zeros[coef : ])).T
print(russian_test[:10])
print(russian_test.shape)

[['Gulamov' '0.0']
 ['Eideman' '0.0']
 ['Filippovich' '0.0']
 ['Baburkin' '0.0']
 ['Bessogonov' '0.0']
 ['Agababoff' '0.0']
 ['Aweritchkin' '0.0']
 ['Glasko' '0.0']
 ['Awinowitski' '0.0']
 ['Baikowski' '0.0']]
(2912, 2)
[['Bezrukikh' '0.0']
 ['Glazanov' '0.0']
 ['Darevsky' '0.0']
 ['Harin' '0.0']
 ['Donchak' '0.0']
 ['Hananaev' '0.0']
 ['Grushikhin' '0.0']
 ['Emelin' '0.0']
 ['Gorsun' '0.0']
 ['Bajutkin' '0.0']]
(729, 2)


In [138]:
english_train = np.vstack((english[:coef], ones[:coef])).T
print(english_train[:10])
print(english_train.shape)
english_test = np.vstack((english[coef: ], ones[coef : ])).T
print(english_test[-10:])
print(english_test.shape)

[['Hunt' '1.0']
 ['Graves' '1.0']
 ['Donohoe' '1.0']
 ['Conroy' '1.0']
 ['Friel' '1.0']
 ['Booth' '1.0']
 ['Arnall' '1.0']
 ['Eglan' '1.0']
 ['Flowers' '1.0']
 ['Allwood' '1.0']]
(2912, 2)
[['Lilwall' '1.0']
 ['Redding' '1.0']
 ['Reeves' '1.0']
 ['Levin' '1.0']
 ['Pearson' '1.0']
 ['Bassett' '1.0']
 ['Mcneil' '1.0']
 ['Keeler' '1.0']
 ['Jenas' '1.0']
 ['Eggison' '1.0']]
(729, 2)


In [139]:
X_train = np.append(russian_train, english_train, axis=0)
X_test = np.append(russian_test, english_test, axis=0)
print(X_train[coef - 10 : coef + 10])
print(X_train.shape)

[['Evert' '0.0']
 ['Eikhenvald' '0.0']
 ['Belosludtsev' '0.0']
 ['Abolins' '0.0']
 ['Avdakoff' '0.0']
 ['Dobretsov' '0.0']
 ['Djunusov' '0.0']
 ['Egamberdiev' '0.0']
 ['Alshits' '0.0']
 ['Abdulhabiroff' '0.0']
 ['Hunt' '1.0']
 ['Graves' '1.0']
 ['Donohoe' '1.0']
 ['Conroy' '1.0']
 ['Friel' '1.0']
 ['Booth' '1.0']
 ['Arnall' '1.0']
 ['Eglan' '1.0']
 ['Flowers' '1.0']
 ['Allwood' '1.0']]
(5824, 2)


Еще раз перемешаем данные

In [140]:
np.random.shuffle(X_train)
np.random.shuffle(X_test)

In [141]:
print(X_train[coef - 10 : coef + 10])

[['Bethell' '1.0']
 ['Perez' '1.0']
 ['Dikson' '0.0']
 ['Rogan' '1.0']
 ['Hamitov' '0.0']
 ['Agar' '1.0']
 ['Gaskov' '0.0']
 ['Amod' '1.0']
 ['Azhogin' '0.0']
 ['Pomphrey' '1.0']
 ['Gudzenko' '0.0']
 ['Besschetny' '0.0']
 ['Ashby' '1.0']
 ['Gladkov' '0.0']
 ['Black' '1.0']
 ['Kay' '1.0']
 ['Doherty' '1.0']
 ['Hairutdinov' '0.0']
 ['Gorchinsky' '0.0']
 ['Thonon' '1.0']]


##  Часть 2. Базовый метод классификации [20 баллов]



Используйте метод наивного Байеса, логистическую регрессию или любой другой метод для классификации фамилий: в качестве признаков используйте символьные $n$-граммы. Сравните результаты, получаемые при разных $n=2,3,4$ по $F$-мере и аккуратности. В каких случаях метод ошибается?

Для генерации $n$-грамм используйте:

In [ ]:
from nltk.util import ngrams


Создадим массив содержащий в строках фамилии в виде биграмм:
- Фамилия - Предложение
- Биграмма - Фамилия в предложении)

In [151]:
from nltk import ngrams
def get_ngrams(train, ngrams_count):
    sequences = []
    for words, label in train[:]:
        seq = list(words.lower())
        bigrams = ngrams(seq, ngrams_count)
        bi = []
        for grams in bigrams:           
            if len(grams) >= ngrams_count:
                bi.append("".join(grams))
        sequences.append([" ".join(bi), int(float(label))])
    return np.array(sequences)

In [179]:
from sklearn.metrics import *
def metrics_test(predicted, target):
    acc = accuracy_score(predicted, target)
    f1_micro = f1_score(predicted, target, average = 'micro')
    f1_macro = f1_score(predicted, target, average = 'macro')
    print('acc={0:1.4f}'.format(acc))
    print('micro F1={0:1.4f}'.format(f1_micro))
    print('macro F1={0:1.4f}'.format(f1_macro))

### n=2

In [180]:
X_train_gram = get_ngrams(X_train, 2)
X_test_gram = get_ngrams(X_test, 2)

In [181]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train_gram[:,0])
X_train_counts.shape

(5824, 510)

In [182]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_counts, X_train_gram[:,1])

In [183]:
X_test_fin = count_vect.transform(X_test_gram[:,0])
predicted = clf.predict(X_test_fin)

In [184]:
metrics_test(predicted, X_test_gram[:,1])

acc=0.9129
micro F1=0.9129
macro F1=0.9129


### n=3

In [185]:
X_train_gram = get_ngrams(X_train, ngrams_count=3)
X_test_gram = get_ngrams(X_test, ngrams_count=3)

In [186]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train_gram[:,0])
X_train_counts.shape

(5824, 3928)

In [187]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_counts, X_train_gram[:,1])

In [188]:
X_test_fin = count_vect.transform(X_test_gram[:,0])
predicted = clf.predict(X_test_fin)

In [189]:
metrics_test(predicted, X_test_gram[:,1])


acc=0.9335
micro F1=0.9335
macro F1=0.9334


### n=4

In [197]:
X_train_gram = get_ngrams(X_train, ngrams_count=4)
X_test_gram = get_ngrams(X_test, ngrams_count=4)

In [198]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train_gram[:,0])
X_train_counts.shape

(5824, 10200)

In [199]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_counts, X_train_gram[:,1])

In [200]:
X_test_fin = count_vect.transform(X_test_gram[:,0])
predicted = clf.predict(X_test_fin)

In [201]:
metrics_test(predicted, X_test_gram[:,1])

acc=0.8903
micro F1=0.8903
macro F1=0.8897


##  Часть 3. Нейронная сеть [35 баллов]


Используйте  реккурентную нейронную сеть с  LSTM для решения задачи. В ней может быть несколько слоев с LSTM, несколько слоев c Bidirectional(LSTM).  У нейронной сети один выход, определяющий класс фамилии. 

Представление имени для классификации в этом случае: 
1 вариант: бинарная матрица размера (количество букв в корпусе $\times$ максимальная длина имени). Обозначим его через $x$. Если первая буква имени a, то $x[1][1] = 1$, если вторая – b, то  $x[2][1] = 1$. То есть, используем one hot encoding для векторизации букв.  
2 вариант: Embedding'и символов. 

Выберите тот вариант, который вам проще или интереснее реализовать :) 

Не забудьте про регуляризацию нейронной сети дропаутами. 

Сравните результаты классификации разными методами по accuracy, micro- и macro- F-measure, precision, recall. Какой метод лучше и почему?

Сравните результаты, получаемые при разных значениях дропаута, разных числах узлов на слоях нейронной сети по $F$-мере и аккуратности. В каких случаях нейронная сеть ошибается?

Если совсем не получается запрограммировать нейронную сеть самостоятельно, обратитесь к туториалу тут: https://github.com/divamgupta/lstm-gender-predictor


Посчитаем максимальные значения для того что бы понять размерности входных данных

In [202]:
totalEntries = X_train.shape[0]
maxlen = len(max( X_train[:, 0] , key=len))

Cоздадим таблицы для перевода символов в индексы и обратно

In [205]:
chars = set(  "".join(X_train[:, 0])  )
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

Посмотрим на размерности

In [212]:
print("total entries " , totalEntries)
print("max len " , maxlen)
print('total chars:', len(chars))

total entries  5824
max len  18
total chars: 54


Подготовим пустые массивы нужной размерности что бы после заполнить их данными

In [214]:
X = np.zeros((totalEntries , maxlen, len(chars) ), dtype=np.bool)
y = np.zeros((totalEntries , 2 ), dtype=np.bool)

Заполним данными подготовленные массивы

In [217]:
for i, (name, sex) in enumerate(X_train):
    for t, char in enumerate(name):
        X[i, t, char_indices[char]] = 1
    y[i, int(float(sex))] = 1

Создадим модель

In [218]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
model = Sequential()
model.add(LSTM(512, return_sequences=True, input_shape=(maxlen, len(chars))))
model.add(Dropout(0.2))
model.add(LSTM(512, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(2))
model.add(Activation('softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam')

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [219]:
model.fit(X, y, batch_size=16, epochs=20)
model.save_weights('my_model_weights.h5')
score = model.evaluate(X, y, batch_size=16)
print("score " , score)

Epoch 1/20
5824/5824 [==============================] - 86s 15ms/step - loss: 0.2894
Epoch 2/20
5824/5824 [==============================] - 86s 15ms/step - loss: 0.2125
Epoch 3/20
5824/5824 [==============================] - 89s 15ms/step - loss: 0.1927
Epoch 4/20
5824/5824 [==============================] - 83s 14ms/step - loss: 0.1836
Epoch 5/20
5824/5824 [==============================] - 82s 14ms/step - loss: 0.1711
Epoch 6/20
5824/5824 [==============================] - 81s 14ms/step - loss: 0.1725
Epoch 7/20
5824/5824 [==============================] - 82s 14ms/step - loss: 0.1474
Epoch 8/20
5824/5824 [==============================] - 82s 14ms/step - loss: 0.1455
Epoch 9/20
5824/5824 [==============================] - 80s 14ms/step - loss: 0.1391
Epoch 10/20
5824/5824 [==============================] - 81s 14ms/step - loss: 0.1331
Epoch 11/20
5824/5824 [==============================] - 80s 14ms/step - loss: 0.1192
Epoch 12/20
5824/5824 [==============================] - 80s 14

Посчитаем максимальные значения для того что бы понять размерности входных данных

In [220]:
totalEntries_t = X_test.shape[0]
maxlen_t = len(max( X_test[:, 0] , key=len))

In [228]:
X_t = np.zeros((totalEntries_t , maxlen_t-1, len(chars) ), dtype=np.bool)
y_t = np.zeros((totalEntries_t , 2 ), dtype=np.bool)

Подготовим тестовые данные

In [229]:
for i, (name, sex) in enumerate(X_test):
    for t, char in enumerate(name):
        try:
            X_t[i, t, char_indices[char]] = 1
        except:
            print("Не найден символ {} в имени: {}".format(char, name))

Не найден символ y в имени: Bekovich-Cherkassky


In [230]:
X_t.shape

(1458, 18, 54)

In [231]:
predicted = model.predict_classes(X_t)

In [232]:
X_tt = X_test[:,1].astype(float).astype(int)

In [233]:
metrics_test(predicted, X_tt)

acc=0.9561
micro F1=0.9561
macro F1=0.9561


Выводы: нейронная сеть дольше обучается но работает лучше. А вот с n-граммами у меня что-то пошло не так. На n=3, результат лучше чем на n=4. Хотя по идеи, чем больше размерность n-грамм, тем лучше результат.

##  Часть 4. Генерация фамилии [35 баллов]

Используйте архитектуру нейронной сети из Части 3 для генерации имени. В этот момент можно забыть про разбиение коллекции на обучающее и тестовое множество. 

Рассмотрите два сценария генерация имени:
* обучение нейронной сети по всей коллекции
* обучение нейронной сети с обуславливанием на язык

Для обуславливания на язык нужно изменить векторное представление фамилии. До этого момента мы разбивали фамилию на отдельные символы и находили векторное представление каждого символа. Теперь добавим в начало фамилии метку языка и будем ее считать первым символов фамилии. Так нейронная сеть научиться понимать, на каком языке написана фамилия. 

Пример: ```[rus bos i v a n o v eos pad pad pad]```

Когда будем генерировать новую фамилию, будем начинать процесс генерации не с символа начала последовательности ```bos```, а с символа языка и символа начала последовательности ```rus bos```. 

Привидите несколько примеров удачно сгенерированных фамилий. 